In [1]:
import os
import sys
import numpy as np
import neuroglancer

from utils import get_structure_number, get_segment_properties, NumpyToNeuroglancer

Connecting dklab@db.dk.ucsd.edu:3306


In [3]:
surround = False
ATLAS_V7_DIR = '/net/birdstore/Active_Atlas_Data/copied_from_S3/mousebrainatlas-data/CSHL_volumes/atlasV7/atlasV7_10.0um_scoreVolume/score_volumes'
files = os.listdir(ATLAS_V7_DIR)
volume_files = sorted([f for f in files if f.endswith('.npy') and surround == ('surround' in f) and 'test' not in f])
origin_files = sorted([f for f in files if f.endswith('.txt') and surround == ('surround' in f) and 'test' not in f])
    
structure_volume_origin = {}
for volume_filename, origin_filename in zip(volume_files, origin_files):
    prefix = os.path.splitext(volume_filename)[0]
    structure = prefix.replace('atlasV7_10.0um_scoreVolume_', '').replace('_surround_200um', '')
    if structure not in origin_filename:
        print(structure, origin_filename)
        break

    color = get_structure_number(structure.replace('_L', '').replace('_R', ''))

    volume = np.load(os.path.join(ATLAS_V7_DIR, volume_filename))
    origin = np.loadtxt(os.path.join(ATLAS_V7_DIR, origin_filename))
    
    volume = np.rot90(volume, axes=(0,1))
    volume = np.flip(volume, axis=0)
    volume[volume > 0.8] = color
    volume = volume.astype(np.uint8)
    
    structure_volume_origin[structure] = (volume, origin)
print(structure_volume_origin.keys())

dict_keys(['10N_L', '10N_R', '12N', '3N_L', '3N_R', '4N_L', '4N_R', '5N_L', '5N_R', '6N_L', '6N_R', '7N_L', '7N_R', '7n_L', '7n_R', 'AP', 'Amb_L', 'Amb_R', 'DC_L', 'DC_R', 'IC', 'LC_L', 'LC_R', 'LRt_L', 'LRt_R', 'PBG_L', 'PBG_R', 'Pn_L', 'Pn_R', 'RMC_L', 'RMC_R', 'RtTg', 'SC', 'SNC_L', 'SNC_R', 'SNR_L', 'SNR_R', 'Sp5C_L', 'Sp5C_R', 'Sp5I_L', 'Sp5I_R', 'Sp5O_L', 'Sp5O_R', 'Tz_L', 'Tz_R', 'VCA_L', 'VCA_R', 'VCP_L', 'VCP_R', 'VLL_L', 'VLL_R'])


In [4]:
x_length = 1000
y_length = 1000
z_length = 300
atlasV7_volume = np.zeros((x_length, y_length, z_length), dtype=np.uint32)

for structure, (volume, origin) in structure_volume_origin.items():
    x, y, z = origin
    x_start = int(x) + x_length // 2
    y_start = int(y) + y_length // 2
    z_start = int(z) // 2 + z_length // 2
    x_end = x_start + volume.shape[0]
    y_end = y_start + volume.shape[1]
    z_end = z_start + (volume.shape[2] + 1) // 2

    z_indices = [z for z in range(volume.shape[2]) if z % 2 == 0]
    volume = volume[:, :, z_indices]
    atlasV7_volume[x_start:x_end, y_start:y_end, z_start:z_end] += volume

In [5]:
OUTPUT_DIR = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/structures/atlasV7'

ng = NumpyToNeuroglancer(atlasV7_volume, [10000, 10000, 20000])
ng.preview()
ng.init_precomputed(OUTPUT_DIR)
ng.add_segment_properties(get_segment_properties())
ng.add_downsampled_volumes()
ng.add_segmentation_mesh()

A new layer named segmentation_[10000, 10000, 20000] is added to:
http://127.0.0.1:35463/v/d4fc3cd26bade930b16d1ed38a0bede7a30a5ea7/


Tasks:   0%|          | 0/5 [00:00<?, ?it/s]

Volume Bounds:  Bbox([0, 0, 0],[1000, 1000, 300], dtype=int32)
Selected ROI:   Bbox([0, 0, 0],[1000, 1000, 300], dtype=int32)



Downloading: 100%|██████████| 256/256 [00:01<00:00, 13.81it/s]

Uploading:   0%|          | 0/64 [00:00<?, ?it/s]

Uploading:   5%|▍         | 3/64 [00:00<00:06,  9.09it/s]

Uploading:   9%|▉         | 6/64 [00:00<00:05, 11.21it/s]

Uploading:  36%|███▌      | 23/64 [00:00<00:02, 20.44it/s]

Uploading:  39%|███▉      | 25/64 [00:00<00:02, 14.84it/s]

Uploading:  58%|█████▊    | 37/64 [00:00<00:00, 56.68it/s]

Uploading:  66%|██████▌   | 42/64 [00:00<00:00, 56.68it/s]

Uploading:   0%|          | 0/64 [00:00<?, ?it/s]

Uploading:  83%|████████▎ | 53/64 [00:00<00:00, 70.86it/s]

Uploading:  91%|█████████ | 58/64 [00:00<00:00, 59.49it/s]

Uploading:  97%|█████████▋| 62/64 [00:00<00:00, 47.69it/s]

Uploading: 100%|██████████| 64/64 [00:00<00:00, 47.69it/s]

Uploading: 100%|██████████| 64/64 [00:00<00:00, 47.69it/s]

Uploading: 100%|██████████| 64/64 [00:00<00:00, 47.69it/s]

Uploading: 100%|██████████| 64/64 [00:00<00:00, 47.69it/s]

Uploading: 100%|██████████| 64/64 [00:00<00:00, 47.69it

Uploading:  50%|█████     | 32/64 [00:00<00:01, 24.64it/s]

Uploading:  34%|███▍      | 22/64 [00:00<00:00, 101.25it/s]

Uploading:  44%|████▍     | 28/64 [00:00<00:00, 101.25it/s]

Uploading:  48%|████▊     | 31/64 [00:00<00:00, 56.89it/s] 

Uploading:  66%|██████▌   | 42/64 [00:00<00:00, 45.26it/s]

Uploading:  69%|██████▉   | 44/64 [00:00<00:00, 37.36it/s]

Uploading:  88%|████████▊ | 56/64 [00:00<00:00, 37.36it/s]

Uploading:  31%|███▏      | 20/64 [00:00<00:00, 86.28it/s]

Uploading:  31%|███▏      | 20/64 [00:00<00:00, 86.28it/s]

Uploading:  33%|███▎      | 21/64 [00:00<00:04,  9.96it/s]

Uploading:  34%|███▍      | 22/64 [00:00<00:06,  6.05it/s]

Uploading:  36%|███▌      | 23/64 [00:00<00:06,  6.05it/s]

Uploading:  39%|███▉      | 25/64 [00:00<00:06,  6.05it/s]

Uploading:  44%|████▍     | 28/64 [00:00<00:05,  6.57it/s]

Uploading:  44%|████▍     | 28/64 [00:00<00:05,  6.57it/s]

Uploading:  48%|████▊     | 31/64 [00:00<00:05,  6.57it/s]

Uploading: 100%|██████████| 64/64 [00

Uploading: 100%|██████████| 64/64 [00:17<00:00,  3.63it/s]

Tasks: 100%|██████████| 999/999 [00:01<00:00, 797.86it/s]


In [ ]:
with open('atlasV7.npy', 'wb') as file:
    np.save(file, atlasV7_volume)